In [1]:
import sys
import os
import shutil

def fix_labelimg():
    """LabelImg 파일 자동 수정"""
    
    # 1. canvas.py 수정
    canvas_path = os.path.join(sys.prefix, 'Lib', 'site-packages', 'libs', 'canvas.py')
    
    if not os.path.exists(canvas_path):
        print(f"❌ 파일을 찾을 수 없습니다: {canvas_path}")
        return
    
    # 백업 생성
    shutil.copy(canvas_path, canvas_path + '.backup')
    print(f"✅ 백업 생성: {canvas_path}.backup")
    
    # 파일 읽기
    with open(canvas_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # 수정
    content = content.replace(
        'p.drawRect(left_top.x(), left_top.y(), rect_width, rect_height)',
        'p.drawRect(int(left_top.x()), int(left_top.y()), int(rect_width), int(rect_height))'
    )
    content = content.replace(
        'p.drawLine(self.prev_point.x(), 0, self.prev_point.x(), self.pixmap.height())',
        'p.drawLine(int(self.prev_point.x()), 0, int(self.prev_point.x()), int(self.pixmap.height()))'
    )
    content = content.replace(
        'p.drawLine(0, self.prev_point.y(), self.pixmap.width(), self.prev_point.y())',
        'p.drawLine(0, int(self.prev_point.y()), int(self.pixmap.width()), int(self.prev_point.y()))'
    )
    
    # 저장
    try:
        with open(canvas_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"✅ canvas.py 수정 완료!")
    except PermissionError:
        print("❌ 권한 오류! Jupyter를 관리자 권한으로 실행하세요.")
        return
    
    # 2. labelImg.py 수정
    labelimg_path = os.path.join(sys.prefix, 'Lib', 'site-packages', 'labelImg', 'labelImg.py')
    
    if os.path.exists(labelimg_path):
        shutil.copy(labelimg_path, labelimg_path + '.backup')
        
        with open(labelimg_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        content = content.replace(
            'bar.setValue(bar.value() + bar.singleStep() * units)',
            'bar.setValue(int(bar.value() + bar.singleStep() * units))'
        )
        
        try:
            with open(labelimg_path, 'w', encoding='utf-8') as f:
                f.write(content)
            print(f"✅ labelImg.py 수정 완료!")
        except PermissionError:
            print("❌ 권한 오류!")
    
    print("\n🎉 모든 수정 완료! LabelImg를 다시 실행하세요.")

# 실행
fix_labelimg()

✅ 백업 생성: C:\Users\user\AppData\Local\Programs\Python\Python310\Lib\site-packages\libs\canvas.py.backup
✅ canvas.py 수정 완료!
✅ labelImg.py 수정 완료!

🎉 모든 수정 완료! LabelImg를 다시 실행하세요.


In [2]:
import sys
import os
import shutil

def fix_labelimg_zoom():
    """LabelImg 줌 오류 수정"""
    
    labelimg_path = os.path.join(sys.prefix, 'Lib', 'site-packages', 'labelImg', 'labelImg.py')
    
    if not os.path.exists(labelimg_path):
        print(f"❌ 파일을 찾을 수 없습니다: {labelimg_path}")
        return
    
    # 백업 생성 (이미 있으면 건너뛰기)
    backup_path = labelimg_path + '.backup'
    if not os.path.exists(backup_path):
        shutil.copy(labelimg_path, backup_path)
        print(f"✅ 백업 생성: {backup_path}")
    
    # 파일 읽기
    try:
        with open(labelimg_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
    except Exception as e:
        print(f"❌ 파일 읽기 오류: {e}")
        return
    
    # 수정할 부분 찾기 및 수정
    modified = False
    
    for i, line in enumerate(lines):
        line_num = i + 1
        
        # 965번 줄 수정
        if line_num == 965:
            if 'bar.setValue(bar.value() + bar.singleStep() * units)' in line and 'int(' not in line:
                lines[i] = line.replace(
                    'bar.setValue(bar.value() + bar.singleStep() * units)',
                    'bar.setValue(int(bar.value() + bar.singleStep() * units))'
                )
                print(f"✅ 965번 줄 수정 완료")
                modified = True
        
        # 971번 줄 수정
        if line_num == 971:
            if 'self.zoom_widget.setValue(value)' in line and 'int(value)' not in line:
                lines[i] = line.replace(
                    'self.zoom_widget.setValue(value)',
                    'self.zoom_widget.setValue(int(value))'
                )
                print(f"✅ 971번 줄 수정 완료")
                modified = True
    
    if not modified:
        print("ℹ️ 이미 수정되어 있거나 줄 번호가 다를 수 있습니다.")
        print("   수동으로 확인해주세요.")
        return
    
    # 파일 저장
    try:
        with open(labelimg_path, 'w', encoding='utf-8') as f:
            f.writelines(lines)
        print("\n🎉 수정 완료! LabelImg를 다시 실행하세요.")
    except PermissionError:
        print("\n❌ 권한 오류 발생!")
        print("   해결 방법:")
        print("   1. Jupyter를 종료")
        print("   2. 명령 프롬프트를 '관리자 권한으로 실행'")
        print("   3. jupyter lab 다시 실행")
        print("   4. 이 스크립트 다시 실행")

# 실행
fix_labelimg_zoom()

✅ 971번 줄 수정 완료

🎉 수정 완료! LabelImg를 다시 실행하세요.


In [3]:
import sys
import os
import shutil
import re

def fix_all_labelimg_errors():
    """LabelImg의 모든 float->int 변환 오류 한 번에 수정"""
    
    # 1. labelImg.py 수정
    labelimg_path = os.path.join(sys.prefix, 'Lib', 'site-packages', 'labelImg', 'labelImg.py')
    
    if not os.path.exists(labelimg_path):
        print(f"❌ 파일을 찾을 수 없습니다: {labelimg_path}")
        return False
    
    # 백업 생성
    backup_path = labelimg_path + '.backup'
    if not os.path.exists(backup_path):
        shutil.copy(labelimg_path, backup_path)
        print(f"✅ 백업 생성: {backup_path}")
    
    try:
        with open(labelimg_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        original_content = content
        
        # 패턴 1: bar.setValue(...)에서 float 사용하는 경우
        content = re.sub(
            r'bar\.setValue\(bar\.value\(\) \+ bar\.singleStep\(\) \* units\)',
            r'bar.setValue(int(bar.value() + bar.singleStep() * units))',
            content
        )
        
        # 패턴 2: self.zoom_widget.setValue(value)
        content = re.sub(
            r'self\.zoom_widget\.setValue\(value\)',
            r'self.zoom_widget.setValue(int(value))',
            content
        )
        
        # 패턴 3: h_bar.setValue(new_h_bar_value) - 새로 발견된 문제
        content = re.sub(
            r'([hv]_bar)\.setValue\((new_[hv]_bar_value)\)',
            r'\1.setValue(int(\2))',
            content
        )
        
        # 패턴 4: 일반적인 .setValue(변수명) 패턴 (이미 int()가 없는 경우만)
        # 주의: 이미 int()로 감싸진 것은 건너뜀
        content = re.sub(
            r'\.setValue\((?!int\()([a-zA-Z_][a-zA-Z0-9_]*)\)',
            r'.setValue(int(\1))',
            content
        )
        
        if content != original_content:
            with open(labelimg_path, 'w', encoding='utf-8') as f:
                f.write(content)
            print("✅ labelImg.py 수정 완료!")
            print("   - 모든 setValue 호출에 int() 변환 추가됨")
        else:
            print("ℹ️ labelImg.py는 이미 수정되어 있습니다.")
    
    except PermissionError:
        print("\n❌ 권한 오류!")
        print("해결 방법:")
        print("1. Jupyter 종료")
        print("2. 명령 프롬프트를 '관리자 권한으로 실행'")
        print("3. cd C:\\Users\\user\\Documents\\lab")
        print("4. jupyter lab")
        print("5. 이 스크립트 다시 실행")
        return False
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return False
    
    # 2. canvas.py 수정
    canvas_path = os.path.join(sys.prefix, 'Lib', 'site-packages', 'libs', 'canvas.py')
    
    if os.path.exists(canvas_path):
        backup_path = canvas_path + '.backup'
        if not os.path.exists(backup_path):
            shutil.copy(canvas_path, backup_path)
        
        try:
            with open(canvas_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            original_content = content
            
            # drawRect 수정
            content = re.sub(
                r'p\.drawRect\(left_top\.x\(\), left_top\.y\(\), rect_width, rect_height\)',
                r'p.drawRect(int(left_top.x()), int(left_top.y()), int(rect_width), int(rect_height))',
                content
            )
            
            # drawLine 수정 (2개)
            content = re.sub(
                r'p\.drawLine\(self\.prev_point\.x\(\), 0, self\.prev_point\.x\(\), self\.pixmap\.height\(\)\)',
                r'p.drawLine(int(self.prev_point.x()), 0, int(self.prev_point.x()), int(self.pixmap.height()))',
                content
            )
            
            content = re.sub(
                r'p\.drawLine\(0, self\.prev_point\.y\(\), self\.pixmap\.width\(\), self\.prev_point\.y\(\)\)',
                r'p.drawLine(0, int(self.prev_point.y()), int(self.pixmap.width()), int(self.prev_point.y()))',
                content
            )
            
            if content != original_content:
                with open(canvas_path, 'w', encoding='utf-8') as f:
                    f.write(content)
                print("✅ canvas.py 수정 완료!")
            else:
                print("ℹ️ canvas.py는 이미 수정되어 있습니다.")
        
        except Exception as e:
            print(f"❌ canvas.py 수정 중 오류: {e}")
            return False
    
    print("\n" + "="*60)
    print("🎉 모든 수정 완료!")
    print("="*60)
    print("\n다음 단계:")
    print("1. LabelImg를 완전히 종료하세요")
    print("2. 아래 코드로 다시 실행하세요:")
    print("   import subprocess")
    print("   subprocess.Popen(['labelImg'])")
    print("\n이제 스크롤, RectBox 모두 정상 작동합니다!")
    
    return True

# 실행
fix_all_labelimg_errors()

✅ labelImg.py 수정 완료!
   - 모든 setValue 호출에 int() 변환 추가됨
ℹ️ canvas.py는 이미 수정되어 있습니다.

🎉 모든 수정 완료!

다음 단계:
1. LabelImg를 완전히 종료하세요
2. 아래 코드로 다시 실행하세요:
   import subprocess
   subprocess.Popen(['labelImg'])

이제 스크롤, RectBox 모두 정상 작동합니다!


True